In [14]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [15]:
food_qa_data = [
    ("What is a good source of vitamin C?", "Oranges are high in vitamin C.", 1),
    ("What is a good source of vitamin C?", "Beef is rich in protein.", 0),
    ("Which food is high in protein?", "Chicken breast is an excellent source of protein.", 1),
    ("Which food is high in protein?", "Apples are sweet and crunchy.", 0),
    ("What's a popular breakfast food?", "Eggs are a common breakfast food.", 1),
    ("What's a popular breakfast food?", "Salmon is often eaten for dinner.", 0),
    ("What is a good source of fiber?", "Oats are a great source of fiber.", 1),
    ("What is a good source of fiber?", "Ice cream is rich in sugar.", 0),
    ("Which food contains healthy fats?", "Avocados are a great source of healthy fats.", 1),
    ("Which food contains healthy fats?", "White bread is low in fat.", 0),
    ("What's a good snack for energy?", "Bananas provide a quick energy boost.", 1),
    ("What's a good snack for energy?", "Lettuce is mostly water and low in energy.", 0),
    ("What food is rich in calcium?", "Dairy products like milk and cheese are rich in calcium.", 1),
    ("What food is rich in calcium?", "Chicken has a lot of protein but not much calcium.", 0),
    ("Which food is low in calories?", "Cucumbers are low in calories and refreshing.", 1),
    ("Which food is low in calories?", "Cheeseburgers are high in calories.", 0),
    ("Which food is a good source of iron?", "Spinach is a great source of iron.", 1),
    ("Which food is a good source of iron?", "Soda contains no iron.", 0),
    ("What food helps in digestion?", "Yogurt with probiotics helps with digestion.", 1),
    ("What food helps in digestion?", "Cookies are high in sugar and do not help with digestion.", 0),
    ("Which fruit is high in antioxidants?", "Blueberries are packed with antioxidants.", 1),
    ("Which fruit is high in antioxidants?", "French fries are high in fat and not in antioxidants.", 0),
    ("Which vegetable is a good source of vitamin A?", "Carrots are a great source of vitamin A.", 1),
    ("Which vegetable is a good source of vitamin A?", "Candy contains no vitamins.", 0),
    ("What food is high in omega-3 fatty acids?", "Salmon is rich in omega-3 fatty acids.", 1),
    ("What food is high in omega-3 fatty acids?", "Chocolate cake is high in sugar, not omega-3.", 0),
    ("Which food is good for bone health?", "Milk is good for bone health due to its calcium content.", 1),
    ("Which food is good for bone health?", "Soft drinks contain no calcium and are not good for bones.", 0),
    ("What food is high in potassium?", "Bananas are high in potassium.", 1),
    ("What food is high in potassium?", "Butter is high in fat but contains little potassium.", 0),
    ("What food helps reduce cholesterol?", "Oats can help reduce cholesterol.", 1),
    ("What food helps reduce cholesterol?", "Fried chicken is high in cholesterol and fat.", 0),
    ("Which food boosts the immune system?", "Citrus fruits like oranges boost the immune system.", 1),
    ("Which food boosts the immune system?", "Candy lowers the immune response due to high sugar.", 0),
    ("Which food is good for eye health?", "Carrots contain beta-carotene, good for eye health.", 1),
    ("Which food is good for eye health?", "Donuts are not beneficial for eye health.", 0),
    ("Which food contains probiotics?", "Kefir is rich in probiotics.", 1),
    ("Which food contains probiotics?", "Potato chips do not contain probiotics.", 0),
    ("Which food is good for heart health?", "Walnuts are good for heart health due to healthy fats.", 1),
    ("Which food is good for heart health?", "Fried foods are bad for heart health.", 0),
    ("What food is high in zinc?", "Pumpkin seeds are high in zinc.", 1),
    ("What food is high in zinc?", "Candy provides no zinc.", 0),
    ("Which food contains a lot of vitamin K?", "Kale is high in vitamin K.", 1),
    ("Which food contains a lot of vitamin K?", "White rice has very little vitamin K.", 0),
    ("Which food is a good source of magnesium?", "Almonds are a good source of magnesium.", 1),
    ("Which food is a good source of magnesium?", "Cake is high in sugar, not magnesium.", 0),
    ("Which fruit is high in vitamin E?", "Avocados are high in vitamin E.", 1),
    ("Which fruit is high in vitamin E?", "Pasta is not a source of vitamin E.", 0),
    ("What food contains folic acid?", "Leafy greens are rich in folic acid.", 1),
    ("What food contains folic acid?", "Soda contains no folic acid.", 0),
    ("Which food helps with hydration?", "Cucumbers are high in water and help with hydration.", 1),
    ("Which food helps with hydration?", "Chips are salty and do not help with hydration.", 0),
    ("Which food is good for brain function?", "Walnuts are good for brain function due to healthy fats.", 1),
    ("Which food is good for brain function?", "Sugary snacks do not aid brain function.", 0),
    ("Which food is high in carbohydrates?", "Pasta is high in carbohydrates.", 1),
    ("Which food is high in carbohydrates?", "Lettuce is low in carbohydrates.", 0),
    ("What food is rich in vitamin B12?", "Clams are an excellent source of vitamin B12.", 1),
    ("What food is rich in vitamin B12?", "Chips have no vitamin B12.", 0),
    ("Which food helps reduce inflammation?", "Turmeric is known to help reduce inflammation.", 1),
    ("Which food helps reduce inflammation?", "Candy can increase inflammation due to sugar.", 0),
    ("What food is high in protein for vegetarians?", "Lentils are a great source of protein for vegetarians.", 1),
    ("What food is high in protein for vegetarians?", "Jelly beans are not a source of protein.", 0),
    ("Which food is rich in vitamin D?", "Fish like salmon are rich in vitamin D.", 1),
    ("Which food is rich in vitamin D?", "Cookies have no vitamin D.", 0),
    ("What food is high in antioxidants?", "Dark chocolate is high in antioxidants.", 1),
    ("What food is high in antioxidants?", "Chips are not a source of antioxidants.", 0),
    ("Which fruit is rich in vitamin A?", "Mangoes are rich in vitamin A.", 1),
    ("Which fruit is rich in vitamin A?", "Fries are not rich in vitamins.", 0),
    ("What food helps with muscle recovery?", "Chicken is a great source of protein for muscle recovery.", 1),
    ("What food helps with muscle recovery?", "Candy contains sugar and is not good for muscle recovery.", 0),
    ("Which food is high in dietary fiber?", "Lentils are high in dietary fiber.", 1),
    ("Which food is high in dietary fiber?", "Cake is not high in fiber.", 0),
    ("What food helps boost energy?", "Oatmeal helps boost energy due to complex carbs.", 1),
    ("What food helps boost energy?", "Chips provide a temporary energy boost but not sustained.", 0),
    ("Which food is a good source of omega-3?", "Flaxseeds are a great source of omega-3.", 1),
    ("Which food is a good source of omega-3?", "Bread does not contain omega-3.", 0),
    ("What food is good for digestive health?", "Probiotic yogurt is good for digestive health.", 1),
    ("What food is good for digestive health?", "Candy is not good for digestive health.", 0),
    ("Which food is low in fat?", "Cucumbers are low in fat.", 1),
    ("Which food is low in fat?", "Fried chicken is high in fat.", 0),
    ("Which food is rich in vitamin B6?", "Bananas are rich in vitamin B6.", 1),
    ("Which food is rich in vitamin B6?", "Candy contains no vitamin B6.", 0),
    ("What food helps with muscle growth?", "Eggs are a great source of protein for muscle growth.", 1),
    ("What food helps with muscle growth?", "Cookies are not helpful for muscle growth.", 0),
    ("Which food is good for skin health?", "Sweet potatoes are good for skin health.", 1),
    ("Which food is good for skin health?", "Pizza is not good for skin health.", 0),
    ("Which food is rich in folate?", "Spinach is rich in folate.", 1),
    ("Which food is rich in folate?", "Soda contains no folate.", 0),
]

class FoodQADataset(Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        question, answer, label = self.data[idx]
        encoding = self.tokenizer.encode_plus(
            question,
            answer,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

def train_food_qa_model(data, model_name="recobo/agriculture-bert-uncased", epochs=3, batch_size=4):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

    train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)
    train_dataset = FoodQADataset(train_data, tokenizer)
    val_dataset = FoodQADataset(val_data, tokenizer)

    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    for epoch in range(epochs):
        model.train()
        for batch in train_dataloader:
            optimizer.zero_grad()
            inputs = {k: v.to(device) for k, v in batch.items() if k != 'label'}
            labels = batch['label'].to(device)
            outputs = model(**inputs, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()

        # Validation
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for batch in val_dataloader:
                inputs = {k: v.to(device) for k, v in batch.items() if k != 'label'}
                labels = batch['label'].to(device)
                outputs = model(**inputs)
                _, predicted = torch.max(outputs.logits, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        print(f"Epoch {epoch+1}/{epochs}")
        print(f"Validation Accuracy: {100 * correct / total:.2f}%")

    return model, tokenizer

In [6]:

# Train the model
trained_model, tokenizer = train_food_qa_model(food_qa_data)

def select_answer(model, tokenizer, question, candidate_answers):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.eval()

    scores = []
    for answer in candidate_answers:
        inputs = tokenizer.encode_plus(
            question,
            answer,
            add_special_tokens=True,
            max_length=128,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        inputs = {k: v.to(device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            score = torch.softmax(logits, dim=1)[0][1].item()  # Probability of being the correct answer
        scores.append(score)

    best_answer_idx = max(range(len(scores)), key=scores.__getitem__)
    return candidate_answers[best_answer_idx], scores[best_answer_idx]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at recobo/agriculture-bert-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
Validation Accuracy: 33.33%
Epoch 2/3
Validation Accuracy: 61.11%
Epoch 3/3
Validation Accuracy: 77.78%


In [8]:
# Example usage
question = "What is a good source of vitamin C?"
candidate_answers = [
    "Oranges are known for their high vitamin C content.",
    "Spinach is rich in iron.",
    "Bananas are a good source of potassium."
]

best_answer, confidence = select_answer(trained_model, tokenizer, question, candidate_answers)
print(f"Question: {question}")
print(f"Best answer: {best_answer}")
print(f"Confidence: {confidence:.4f}")

Question: What is a good source of vitamin C?
Best answer: Bananas are a good source of potassium.
Confidence: 0.9924


In [ ]:
#Agriculture related example